In [15]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [2]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMTUyM2E3M2YtZmY0My00OTQwLWJlNzktMzlmNjQ2N2QzMzM3IiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAicmVjaXBpZW50S2V5cyI6IFsiMjJMUXk3ZjN0cnRIZ1pvUlZtdW1YR0E0Q3pFaGhkVHJzejV5c3pidHBjeEciXX0=


In [3]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYTVjMWUxMDQtMTRlYy00ODg4LTg3MjgtYzUyZTI0MmVlNGJjIiwgInJlY2lwaWVudEtleXMiOiBbIkJ2MnNtZWEyYVQ1dFNocjlheUFjU1FwS3NoQzFWc3VMaGpKb0pDem1ZQ0tRIl0sICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QifQ==


In [4]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': 'a5c1e104-14ec-4888-8728-c52e242ee4bc',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['Bv2smea2aT5tShr9ayAcSQpKshC1VsuLhjJoJCzmYCKQ'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [5]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 3uZMpXzs9b8KMfTzrHMi8SQf9eySzsSqCQCgU3aUTeyF5axUWomFPPzkhtjcG93pLmEaGaXvfpcrmGGpBkNvXEBB

Public key: 
 Tz6sMt7RVPFgfrGALgqLQSgqWQp24SC8bHQSeETh2mf


**Sending a connection request to acapy**

In [6]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [7]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [8]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

### FEATURE DISCOVERY

In [9]:
# pp.pprint(util.getFeatureDiscovery(agents[0]))

In [35]:
payload = "some string yes dddd"
payload_response = "response yay"
hashid = hashlib.sha256(payload.encode("UTF-8")).hexdigest()
hashid_response = hashlib.sha256(payload_response.encode("UTF-8")).hexdigest()
print(payload, hashid)

some string yes dddd 903add166c6526429fc224bdfa339c99ea1069b9d0e2c39c42a1fd6e0dc2b630


In [19]:
## TODO: this should return both creds

connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

df4ca9d6-560c-46f4-b1cf-eb8da6f37510 
 1247c7cf-18bc-44c3-b362-99a6bb3a10c5


In [46]:
send = requests.post('http://localhost:8151/schema-exchange/send', json={
  "payload": "asdad",
  "connection_id": connection[1]
})

In [47]:
send = json.loads(send.text)
print(send)

{'payload': 'asdad', 'connection_id': '1247c7cf-18bc-44c3-b362-99a6bb3a10c5', 'hashid': '9321dba76b029b298ee76fbb3e25f2a6261806d62b7c03b3754bafd5737b5f45'}


In [50]:
get = requests.get('http://localhost:8151/schema-exchange/' + send['hashid'])
print(send['hashid'])
#     get = json.loads(get.text)
print(get)

9321dba76b029b298ee76fbb3e25f2a6261806d62b7c03b3754bafd5737b5f45
<Response [404]>


In [100]:
send = requests.post('http://localhost:8150/schema-exchange/send-response', json={
  "payload": "response yay",
  "connection_id": connection[1],
    "state": "accepted",
    "hashid": get['hashid']
})
response = json.loads(send.text)

In [101]:
response

{'payload': 'response yay',
 'connection_id': '75878b2d-f674-48e9-9062-878063013548',
 'state': 'accepted'}

In [103]:
send = requests.get('http://localhost:8151/schema-exchange/' + hashid_response)
print(send.text)

{"hashid": "1ed513da69bd4484dca947eb42d87a55d9842ae6d3634fb3a595aefa6c1fd0f5", "payload": "response yay", "state": "accepted", "author": "other", "connection_id": "75878b2d-f674-48e9-9062-878063013548"}


In [31]:
# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [32]:
# newDid = util.createDid(agents[0])
# pp.pprint(newDid['result'])

In [33]:
# util.registerDidOnLedger(agents[0], newDid['result']['did'], newDid['result']['verkey'])

In [34]:
# util.setPublicDid(agents[0], newDid['result']['did'])

In [35]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [36]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [37]:
# util.schemaGet(agents[0], last_schema_id)